In [1]:
# w제곱한거 누적
# weight가 높을수록 (1에 가까워질수록) 비슷한 애들이 나옴

In [ ]:
# [1시간] -> multinomial

# section/100 101 102 ...
# https://n.news.naver.com/article/022/0003927165
#     102-0003927165.txt 
# 1. 분류코드를 부착한 뉴스 수집기
# 2. section 값을 얻어야함
# 3. Indexing
# 4. K-retreival => kNN (7)
# 5. 뒷부분 vector space 부분에서 분류하는 것
# 6. 

# [스팸분류기] -> binomial

In [95]:
import re
from requests.compat import urljoin
import requests
from bs4 import BeautifulSoup

In [97]:
section = list()
url = "https://news.naver.com"
path = "./news/"

resp = requests.get(url)
dom = BeautifulSoup(resp.text, 'html.parser')
for a in dom.select('.Nlnb_menu_list > li > a[href]')[1:7]:
    nurl = urljoin(url, a.attrs['href'])
    section.append(re.search(r'(\d{3})$', nurl).group(1))

dic = dict()

for num in section:
    url = 'https://news.naver.com/section/' + str(num)
    resp = requests.get(url)
    dom = BeautifulSoup(resp.text, 'html.parser')
    title = dom.select('div.sa_thumb_inner > a')
        # news link collect
    temp = list()
    
    for t in range(len(title)):
        if title[t]['href'] not in temp:
            temp.append(title[t]['href'])
            resp = requests.get(title[t]['href'])
            dom = BeautifulSoup(resp.text, 'html.parser')
            
            with open('./news/'+str(num)+'-'+title[t]['href'].split('/')[-1], 'w') as f:
                resp = requests.get(title[t]['href'])
                dom = BeautifulSoup(resp.text, 'html.parser')
                f.write(dom.get_text())

KeyboardInterrupt: 

In [99]:
from os import listdir

def fileids(path):
    return [path - ('/' if path[-1] != '/' else '') + f for f in listdir(path) if re.search(r'\d+[-]\d+[.]txt$', f)]

In [ ]:
from requests import get
from requests.compat import urljoin
from bs4 import BeautifulSoup
import re

urls = [('https://news.naver.com', 0)]
seens = []
path = './news/'

while urls:
    url = urls.pop(0)
    
    resp = get(url[0])
    seens.append(url[0])
    
    if not re.search(r'text\/html', resp.headers['content-type']):
        continue
        
    dom = BeautifulSoup(resp.text, 'html.parser')
    
    # 1번 - 뉴스 메뉴
    for a in dom.select('.Nlnb_menu_list > li > a[href]')[1:7]:
        nurl = urljoin(url[0], a.attrs['href'])
        if nurl not in urls and nurl not in seens:
            urls.append((nurl, seens.index(url[0])))
            
    # 2번 - 헤드라인
    for a in dom.select('[id^="_SECTION_HEADLINE_LIST"] .sa_text_title[href]'):
        nurl = urljoin(url[0], a.attrs['href'])
        if nurl not in urls and nurl not in seens:
            urls.append((nurl, seens.index(url[0])))
            
    # 3번 - 뉴스본문
    c = dom.select_one('#contents')
    if c:
        section = re.search(r'(\d{3})$', seens[url[1]]).group(1)
        file = re.search(r'(\d{8,})$', url[0]).group(1)
        with open(f'{path}{section}-{file}.txt', 'w', encoding='utf8') as f:
            f.write(c.get_text())